## Importing libraries, loading dataset,checking it's info and  removing 341 null values from kind column

In [44]:
import numpy as np
import pandas as pd
from datetime import datetime
import json
from pandas.tseries.offsets import MonthEnd

In [2]:
# Loading the dataset
df=pd.read_csv('Data Sheet.csv')

In [3]:
# Checking the dataset and it's column 
df.head(2)

,_id,created_at,_v,_fivetran_deleted,milestones,booking_id,updated_at,_fivetran_synced,kind,organization_id,...,status,customer_invoice_no,ticket_status,zendesk_ticket_id,zoho_invoice_status,zoho_estimate_invoice_nos,container_code,flight_number,airline_name,carrier_code
0,63b3b00812fd293bff52248d,2023-01-03 04:33:12.157+00,0,False,"[{""dateTime"":""2023-01-03T10:25:00Z"",""details"":...",63b3b00812fd293bff522488,2023-07-20 07:24:41.662+00,2023-07-20 08:18:03.235+00,NaN,NaN,...,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,63b50769ba169e7e19712077,2023-01-04 04:58:17.462+00,0,False,"[{""dateTime"":""2023-01-01T22:31:00Z"",""details"":...",63b50769ba169e7e19712072,2023-07-20 07:24:41.732+00,2023-07-20 08:18:03.236+00,NaN,NaN,...,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df.shape

(1964, 22)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1964 entries, 0 to 1963
Data columns (total 22 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   _id                        1964 non-null   object 
 1   created_at                 1964 non-null   object 
 2   _v                         1964 non-null   int64  
 3   _fivetran_deleted          1964 non-null   bool   
 4   milestones                 1964 non-null   object 
 5   booking_id                 1964 non-null   object 
 6   updated_at                 1964 non-null   object 
 7   _fivetran_synced           1964 non-null   object 
 8   kind                       1623 non-null   object 
 9   organization_id            1623 non-null   object 
 10  quote_identity             1623 non-null   object 
 11  booking_identity           1623 non-null   object 
 12  status                     473 non-null    object 
 13  customer_invoice_no        1503 non-null   objec

In [6]:
# Ckecking the null values present in the dataset
df.isnull().sum()

_id                             0
created_at                      0
_v                              0
_fivetran_deleted               0
milestones                      0
booking_id                      0
updated_at                      0
_fivetran_synced                0
kind                          341
organization_id               341
quote_identity                341
booking_identity              341
status                       1491
customer_invoice_no           461
ticket_status                  19
zendesk_ticket_id            1184
zoho_invoice_status          1777
zoho_estimate_invoice_nos    1777
container_code               1539
flight_number                1583
airline_name                 1285
carrier_code                 1571
dtype: int64

In [29]:
# filtering out all rows in the DataFrame where the 'kind' column has null values, effectively removing incomplete data based on that column.
df = df[df['kind'].notnull()]

In [30]:
# Checking if the 341 null values are removed from the column kind
df.isnull().sum()

_id                             0
created_at                      0
_v                              0
_fivetran_deleted               0
milestones                      0
booking_id                      0
updated_at                      0
_fivetran_synced                0
kind                            0
organization_id                 0
quote_identity                  0
booking_identity                0
status                       1452
customer_invoice_no           237
ticket_status                   3
zendesk_ticket_id            1097
zoho_invoice_status          1473
zoho_estimate_invoice_nos    1473
container_code               1358
flight_number                1267
airline_name                  969
carrier_code                 1379
dtype: int64

### Average delivery time for each product type (fcl,lcl, etc) for each month in 2023
### tp90 delivery time for each product type (fcl,lcl, etc) for each month in 2023
###  No. of customers with at least 1 delivery

In [47]:
# Function to extract the relevant dates from the Milestones column
def extract_dates(milestones):
    booking_confirmed_date = None
    delivery_date = None
    
    # Parse the JSON milestones column
    milestones = json.loads(milestones)
    
    # Loop through each milestone and extract relevant dates
    for milestone in milestones:
        if milestone['value'] == 'Booking Confirmed' and milestone['dateTime']:
            booking_confirmed_date = milestone['dateTime']
        elif milestone['value'] == 'Delivered to Consignee' and milestone['dateTime']:
            delivery_date = milestone['dateTime']
    
    return booking_confirmed_date, delivery_date

# Apply the function to extract the relevant dates
df[['Booking Confirmed Date', 'Delivered Date']] = df['milestones'].apply(lambda x: pd.Series(extract_dates(x)))

# Convert the extracted date strings to datetime format
df['Booking Confirmed Date'] = pd.to_datetime(df['Booking Confirmed Date'], errors='coerce', utc=True)
df['Delivered Date'] = pd.to_datetime(df['Delivered Date'], errors='coerce', utc=True)

# Calculate delivery time in days
df['Delivery Time'] = (df['Delivered Date'] - df['Booking Confirmed Date']).dt.days

# Filter out records where 'Kind' (product type) is null and where the year is not 2023
df = df[df['kind'].notnull()]
df = df[df['Delivered Date'].dt.year == 2023]

# Filter records where either date is missing (NaT values)
df = df.dropna(subset=['Booking Confirmed Date', 'Delivered Date'])

# Extract month and year (for grouping) and ensure the data is for 2023
df['Year-Month'] = df['Delivered Date'].dt.to_period('M')

# Group by Year-Month and Kind (product type) for 2023 data
grouped = df.groupby(['Year-Month', 'kind'])

# Calculate the average delivery time and tp90 (90th percentile)
summary = grouped['Delivery Time'].agg(
    Average_Delivery_Time='mean',
    TP90_Delivery_Time=lambda x: np.percentile(x, 90)
).reset_index()

# Create a complete list of all Year-Months for 2023 and all product types (Kind)
all_months = pd.period_range('2023-01', '2023-12', freq='M')
all_kinds = df['kind'].unique()

# Create a MultiIndex DataFrame for every combination of month and product type (Kind)
complete_index = pd.MultiIndex.from_product([all_months, all_kinds], names=['Year-Month', 'kind'])
complete_df = pd.DataFrame(index=complete_index).reset_index()

# Merge the complete month-product combinations with the calculated summary
summary_complete = pd.merge(complete_df, summary, on=['Year-Month', 'kind'], how='left')

# Fill NaN values in 'Average_Delivery_Time' and 'TP90_Delivery_Time' with 0 where necessary
summary_complete['Average_Delivery_Time'].fillna(0, inplace=True)
summary_complete['TP90_Delivery_Time'].fillna(0, inplace=True)

# Calculate the number of customers with at least 1 delivery in 2023
# Assuming 'Customer ID' column is available in the dataset
customer_counts = df.groupby('_id').size().reset_index(name='Delivery Count')
customer_counts = customer_counts[customer_counts['Delivery Count'] > 0]
unique_customers_2023 = customer_counts['_id'].nunique()

# Output the results
print("Average Delivery Time and TP90 per Product Type per Month (2023 only, with zeros included):")
print(summary_complete)

print("\nNumber of customers with at least 1 delivery in 2023:")
print(unique_customers_2023)

output_file = 'delivery_time_summary_2023.xlsx'

with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
    summary_complete.to_excel(writer, sheet_name='Delivery Summary', index=False)
    customer_counts.to_excel(writer, sheet_name='Customer Counts', index=False)

print(f'Results have been saved to {output_file}')

C:\Users\vipul\AppData\Local\Temp\ipykernel_19056\4209534209.py:36: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  df['Year-Month'] = df['Delivered Date'].dt.to_period('M')
C:\Users\vipul\AppData\Local\Temp\ipykernel_19056\4209534209.py:59: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  summary_complete['Average_Delivery_Time'].fillna(0, inplace=True)
C:\Users\vipul\AppData\Local\Temp\ipykernel_19056\4209534209.py:60: FutureWarning: A value is trying to be set on a copy of a DataFr

Average Delivery Time and TP90 per Product Type per Month (2023 only, with zeros included):
   Year-Month                  kind  Average_Delivery_Time  TP90_Delivery_Time
0     2023-01          fcl-tracking               0.000000                 0.0
1     2023-01  air_freight-tracking               0.000000                 0.0
2     2023-01          lcl-tracking               0.000000                 0.0
3     2023-01           dd-tracking               0.000000                 0.0
4     2023-01     trucking-tracking               0.000000                 0.0
5     2023-02          fcl-tracking               0.000000                 0.0
6     2023-02  air_freight-tracking               0.000000                 0.0
7     2023-02          lcl-tracking               0.000000                 0.0
8     2023-02           dd-tracking               0.000000                 0.0
9     2023-02     trucking-tracking               0.000000                 0.0
10    2023-03          fcl-tracking    

In [49]:
# Save to Excel
with pd.ExcelWriter('delivery_summary_2023.xlsx') as writer:
    # Save the summary of delivery times
    summary_complete_df.to_excel(writer, sheet_name='Delivery_Summary', index=False)
    
    # Save the number of unique customers
    unique_customers_df.to_excel(writer, sheet_name='Unique_Customers', index=False)

print("Data has been successfully written to 'delivery_summary_2023.xlsx'")


Data has been successfully written to 'delivery_summary_2023.xlsx'
